In [1]:
%pip install av

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
import pandas
import os
import matplotlib.pyplot as plt
from PIL import Image
import torchvision
from torch.utils.data import Dataset, DataLoader
from moviepy.editor import VideoFileClip, vfx
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import moviepy
import tensorflow as tf
import av
import cv2

error: XDG_RUNTIME_DIR not set in the environment.
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM default
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evalu

In [3]:
# data = []
# path = ''
# numbers = os.listdir(path)
# for number in numbers:
#     names = os.listdir(path + number)[:100]
#     for name in names:
#         im_frame = Image.open(f'{path}/{number}/{name}')
#         np_frame = np.array(im_frame.getdata()).reshape(3, 1920, 1080)
#         data.append(np_frame)

# data = np.array(data)

In [4]:
# class Dataset(Dataset):
#     def __init__(self, data):
#         self.data = torch.tensor(data / 255).type(torch.float)

#         labels = []
#         for i in range(10):
#             for j in range(int(len(data) / 10)):
#                 labels.append(i)
#         self.labels = labels

#     def __getitem__(self, idx):
#         return self.data[idx], self.labels[idx]

#     def __len__(self):
#         return len(self.data)

In [5]:
# dataset = Dataset(data)
# dataloader = DataLoader(dataset, shuffle=True, batch_size = 16)

# samples, labels = next(iter(dataloader))

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv3d(in_channels=3,out_channels=32, kernel_size=5,padding=2, stride=1)
        self.conv2 = nn.Conv3d(in_channels=32,out_channels=64, kernel_size=3,padding=1, stride=1)
        self.conv3 = nn.Conv3d(in_channels=64,out_channels=128, kernel_size=3,padding=1, stride=2, dilation=2)
        self.conv4 = nn.Conv3d(in_channels=128,out_channels=256, kernel_size=3,padding=1, stride=2)
        self.conv5 = nn.Conv3d(in_channels=256,out_channels=512, kernel_size=1,padding=1, stride=2, dilation=1)
        self.conv6 = nn.Conv3d(in_channels=512,out_channels=1024, kernel_size=1,padding=1, stride=2,  dilation=1)
        self.pool1 = nn.MaxPool3d(2,2)
        self.pool2 = nn.MaxPool3d(1,1)
        self.fc1 = nn.Linear(24576, 10000)
        self.fc2 = nn.Linear(10000, 500)
        self.fc3 = nn.Linear(500, 3)
        self.seq1 = x = nn.Sequential(self.conv1, self.pool1, nn.ReLU(), 
                          self.conv2, self.pool1, nn.ReLU(),
                          self.conv3, self.pool2, nn.ReLU()
                          )
        self.seq2 = nn.Sequential(self.conv4, self.conv5, self.conv6, self.pool2,  nn.Flatten(1))


    def forward(self, x):
        x = self.seq1(x)
        x = self.seq2(x)
        x = x.view(x.size(0),-1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [7]:
# torch.cuda.empty_cache()

# b = torch.zeros(8, 3, 15, 64, 128).to('cuda')
# # b = torch.zeros(batch_size=8, chanels=3, frames=5, height=64, weight=128).to('cuda')

In [40]:
def split_video_frame(video_path):
    cap = cv2.VideoCapture(video_path)

    frame_width = int(cap.get(3))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out_left = cv2.VideoWriter('left_half.mp4', fourcc, 25.0, (frame_width//2, 1080))
    out_right = cv2.VideoWriter('right_half.mp4', fourcc, 25.0, (frame_width//2, 1080))

    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            left_frame = frame[:, :frame_width//2]
            right_frame = frame[:, frame_width//2:]

            out_left.write(left_frame)
            out_right.write(right_frame)
        else:
            break

    cap.release()
    out_left.release()
    out_right.release()
    cv2.destroyAllWindows()

inputs_right = []
inputs_left = []
split_video_frame('learn.mp4')


def video_to_tensor(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
     
    while(cap.isOpened()):
        ret, frame = cap.read()
         
        if not ret:
            break
         
        frame = cv2.resize(frame, (64, 128))
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        frames.append(frame)
         
    cap.release()
     
    frames = frames[:5]
    while len(frames) < 5:
        frames.append(np.zeros((64, 128, 3), dtype=np.uint8))
     
    tensor = np.array(frames)
    tensor = np.transpose(tensor, (0, 3, 1, 2))
    
    tensor = np.expand_dims(tensor, 0)
    tensor = np.repeat(tensor, 8, axis=0)
    
    tensor = torch.from_numpy(tensor)
     
    return tensor


right_video_path = "right_half.mp4"
rigth_video = moviepy.editor.VideoFileClip(right_video_path)
right_duration = int(rigth_video.duration)
left_video_path = "left_half.mp4"
left_video = moviepy.editor.VideoFileClip(left_video_path)
left_duration = int(left_video.duration)
for i in range(0, right_duration, 5):
    start = i
    end = (i + 5) if (i + 5) < right_duration else right_duration
    output = f"output_{start}_{end}.mp4"
    ffmpeg_extract_subclip(right_video_path, start, end, targetname=output)
    inputs_right.append(video_to_tensor(output))

# for i in range(0, left_duration, 5):
#     start = i
#     end = (i + 5) if (i + 5) < left_duration else left_duration
#     output = f"output_{start}_{end}.mp4"
#     ffmpeg_extract_subclip(left_video_path, start, end, targetname=output)
#     inputs_left.append(video_to_tensor(output))

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join

In [41]:
inputs_right[0].shape

torch.Size([8, 5, 3, 128, 64])

In [42]:
labels = [1] * 12 + [0] * 12 + [2] * 12

In [43]:
inputs_right = inputs_right[:36]

In [44]:
# torch.cuda.empty_cache()

# b = torch.zeros(8, 3, 15, 64, 128).to('cuda')

In [45]:
np.transpose(inputs_right[0], (0, 2, 1, 4, 3)).shape

torch.Size([8, 3, 5, 64, 128])

In [46]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class VideoDataset(Dataset):
    def __init__(self, inputs_right, labels):
        self.inputs_right = inputs_right
        self.labels = labels

        self.length = len(inputs_right)

    def __getitem__(self, index):
        return self.inputs_right[index], self.labels[index]

    def __len__(self):    
        return self.length

dataset = VideoDataset(np.array(inputs_right), np.array(labels))

dataloader = DataLoader(dataset, shuffle=True, batch_size = 2)

model = Net().to(device)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 10

for epoch in range(epochs):
    running_loss = 0.0
    batch_count = 0
    for inputs, targets in dataloader:
        optimizer.zero_grad()
        outputs = model(np.transpose(inputs, (0, 2, 1, 4, 3)))
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        batch_count += 1
    print(f"Epoch {epoch+1}, loss: {running_loss / batch_count}")

torch.save(model, "model.pth")

ValueError: axes don't match array

In [ ]:
# torch.cuda.empty_cache()

# model = Net()
# model.cuda()
# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


# model(torch.tensor(dataloader))

In [ ]:
import gc
gc.collect()